In [ ]:
#Circle bundle imports
import circle_bundles as cb
from circle_bundles.metrics import RP1UnitVectorMetric as rp1_metric

#Persistent homology computations
from ripser import ripser
from persim import plot_diagrams

#For generating optical flow patch samples and computing predominant directions
from synthetic import sample_opt_flow_torus


#For visualizing optical flow patches
import matplotlib.pyplot as plt
from optical_flow.patch_viz import make_patch_visualizer

# Optical Flow Torus Model

In [ ]:
#Generate a sampling of the optical flow torus model over RP1
n_samples = 5000
n = 3
sample_r = False
data, base_points, alpha, r = sample_opt_flow_torus(n_samples, dim = n, sample_r = sample_r)

print(f'{n_samples} optical flow patches generated.')

In [ ]:
#View a sample of the data
patch_vis = make_patch_visualizer()
fig = cb.show_data_vis(data, patch_vis, sampling_method = None, max_samples = 30)
plt.show()

In [ ]:
#Show a PCA visualization of the dataset
cb.show_pca(data)

In [ ]:
#Run Ripser on a sample of the data

dgms_2 = ripser(data, coeff=2, maxdim=2, n_perm=500)["dgms"]
dgms_3 = ripser(data, coeff=3, maxdim=2, n_perm=500)["dgms"]

# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)

plot_diagrams(dgms_2, ax=axes[0], title="coeff = 2")
plot_diagrams(dgms_3, ax=axes[1], title="coeff = 3")

plt.tight_layout()
plt.show()


In [ ]:
#Construct a cover of the base space
n_landmarks = 12
lmk_angs = np.linspace(0, np.pi,n_landmarks, endpoint= False)
landmarks = np.array([np.cos(lmk_angs), np.sin(lmk_angs)]).T
overlap = 1.99

radius = overlap* np.pi/(2*n_landmarks)

cover = cb.MetricBallCover(base_points, landmarks, radius, metric = rp1_metric())
cover_data = cover.build()
summ = cover.summarize(plot = True)

In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
from dreimac import CircularCoords

bundle = cb.build_bundle(
    data,
    cover,
    CircularCoords_cls=CircularCoords,
    show=True,
)


In [ ]:
#Compute class persistence on the weights filtration of the nerve
pers = bundle.get_persistence(show = True)


In [ ]:
#Get a global coordinatization compatible with the maximal subcomplex of the nerve on which 
#the characteristic class representatives are coboundaries 

triv_result = bundle.get_global_trivialization()
print('Global coordinates computed.')

In [ ]:
#Show an interactive visualization of the bundle colored by fiber coordinate
cb.attach_bundle_viz_methods()

fig = bundle.show_bundle(colors = triv_result.F)
plt.show()

In [ ]:
#Show the correlations between local circular coordinates on overlaps
fig = bundle.compare_trivs()
plt.show()

In [ ]:
#Show a visualization of the nerve labeled with SW1

#Compute a potential for the restricted orientation class
subcomplex = bundle.get_max_trivial_subcomplex()
edges = subcomplex.kept_edges
Omega = bundle.classes.cocycle_used.restrict(edges)
phi_vec = Omega.orient_if_possible(edges)[2]
phi = {lmk: phi_vec[lmk] for lmk in range(n_landmarks)}
omega = bundle.classes.omega_O1_used

fig = bundle.show_circle_nerve(omega = omega, phi = phi)
plt.show()

In [ ]:
#Show a recovered patch diagram
per_row = 5
per_col = 9
predom_dirs = np.arctan2(base_points[:,1], base_points[:,0]) % np.pi
coords = np.column_stack([predom_dirs.reshape(-1,1), triv_result.F.reshape(-1,1)])

fig = cb.lattice_vis(
    data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=19,
    thumb_px=350,   
    dpi=350         
)

plt.show()
